Required Packages

In [1]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import math
import re
from collections import Counter
from difflib import SequenceMatcher
import unicodedata
import time
from snowflake_utils import snowflake_utils as sf_utils
sf_helper = sf_utils.SnowflakeHelper()
sf_engine = sf_helper.get_sqlalchemy_engine()
version='v1'

/opt/conda/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/opt/conda/lib/python3.7/site-packages/snowflake/connector/options.py:44: UserWarning: You have an incompatible version of 'pyarrow' installed, please install a version that adheres to: 'pyarrow<0.18.0,>=0.17.0; extra == "pandas"'
  warn_incompatible_dep('pyarrow', _installed_pyarrow.version, _expected_version)


SQL DATA Preparation

In [2]:
# file = open("ent_sqlcode.txt")
# line = file.read()
# file.close()

In [3]:
# def PreSql(queries):
#     start = time.process_time()
#     for i in line.strip().split(';'):
#         print(sf_helper.execute(i))
#         print(time.process_time()-start)
        
# PreSql(line)    

Data Loading from SnowFlake

In [4]:
query = """
select * from  edw_dna_db.wi.sirajmoh_NON_ENT_alt_unmapped 
where GU_CUST_MKT_SEGMENT_CD = 'COM' and ISO_COUNTRY_CODE = 'USA'
"""
query2 = """
select distinct RIGHT(CONCAT('000000', CAST(DUNSNUMBER AS VARCHAR(16))), 9) as dunsnumber, A.country,
 B.iso_3_code, city, state, address, company from
 "EDW_DNA_DB"."WI"."HGDATA_DUNS" A
 left join
 edw_dna_db.wi.sirajmoh_country_iso_codes B
 on A.country = B.country
"""
query3 = """
select distinct DUNSNUMBER from edw_dna_db.wi.hgdata_Tot_ENT_gu_match;
"""

In [5]:
remove_words = ["LIMITED","CORP","PRIVATE","PVT","LTD","COMPANY","GMBH",'LLC',
                "SERVICES","LLP","CORPORATION","LTDA",
               "INC","CO","PTE","PLC","THE","LIMITED",'PT','SPA','SRL','ITALIA','PTY']

pat = r'\b(?:{})\b'.format('|'.join(remove_words))

# remove_count= ["INDIA","ITALIA","AUSTRALIA","NIGERIA","FRANCE",,"CANADA","SINGAPORE",
#"UK","USA","GERMANY","AMERICA","AMERICANS","SAUDI ARABIA","INDONESIA",'TECHNICAL','SOLUTIONS']

In [6]:
def clean_letters(text):
    clean_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore')
    return clean_text
def DataPreprocessing(query,query2):
    df = sf_helper.get_df(query)
    df_cust = sf_helper.get_df(query2) 
    df.drop_duplicates(subset=['gu_party_id','gu_primary_name'],inplace=True)
    df.dropna(subset=['primary_name'],inplace=True)
    df_cust.dropna(subset=['company'],inplace=True)
    df.fillna('None',inplace=True)
    df_cust.fillna('None',inplace=True)
    df.rename(columns={"primary_name": "company_name"},inplace=True)
    columns1= ['company_name','line_1_address', 'city_name', 'state_or_province_name','iso_country_code']
    for i in columns1:
        df[i] = df[i].str.upper()
    columns2= ['company', 'address', 'city', 'state','country','iso_3_code']
    for i in columns2:
        df_cust[i] = df_cust[i].str.upper()
    df['company_name'] = df['company_name'].str.replace(pat, ' ')
    df_cust['company'] = df_cust['company'].str.replace(pat, ' ')
    df['company_name'] = df['company_name'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
    df_cust['company'] = df_cust['company'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
    #Convert white spaces into nulls
    df.company_name.replace(r'^\s*$', np.nan, regex=True,inplace=True)
    df_cust.company.replace(r'^\s*$', np.nan, regex=True,inplace=True)
    for i in columns1:
        df[i] = df[i].apply(lambda x: clean_letters(str(x))).str.decode("utf-8")
    for i in columns2:
        df_cust[i] = df_cust[i].apply(lambda x: clean_letters(str(x))).str.decode("utf-8")
    df.dropna(subset=['company_name'],inplace=True)
    df_cust.dropna(subset=['company'],inplace=True)
    return df,df_cust

In [7]:
#Step 1
df,df_cust = DataPreprocessing(query,query2)

In [8]:
df.shape

(1847427, 11)

In [9]:
df.head()

,gu_party_id,gu_primary_name,duns_number,gu_cust_mkt_segment_cd,alt_dunsnumber,company_name,branch_level,iso_country_code,line_1_address,city_name,state_or_province_name
0,348743600,FARM FINANCIAL HOLDINGS LLC,-999,COM,-999,FARM FINANCIAL HOLDINGS,BR,USA,1033 DEMONBREUN ST,NASHVILLE,TN
1,150253613,HORIZON COMPUTER SERVICES,807276253,COM,807276253,HORIZON COMPUTER,BR,USA,16301 RABON CHAPEL RD,MONTGOMERY,TX
2,7571825,WORLD HQ LTD,-999,COM,-999,WORLD HQ,BR,USA,200 INTERNATIONAL CIR,COCKEYSVILLE,MD
3,37956855,TOTAL TECHNOLOGY INTEGRATION,147076975,COM,147076975,TOTAL TECHNOLOGY INTEGRATION,HQ,USA,109 GEORGIAN DR,BUTLER,PA
4,303061366,PAUL NUNCHUCK AND RONNIE KELLY,-999,COM,-999,PAUL NUNCHUCK AND RONNIE KELLY,BR,USA,PAUL NUNCHUCK,JACKSONVILLE,FL


In [10]:
df.gu_party_id.nunique()

1847425

In [11]:
df_duns_mapped_ENT = sf_helper.get_df(query3)

In [12]:
df_duns_mapped_ENT.shape

(91966, 1)

In [13]:
df_duns_mapped_ENT['dunsnumber'] = df_duns_mapped_ENT['dunsnumber'].astype(str)

In [14]:
df_duns_mapped_ENT.shape

(91966, 1)

In [15]:
df_cust.shape

(2417355, 7)

In [16]:
df_cust.dtypes

dunsnumber    object
country       object
iso_3_code    object
city          object
state         object
address       object
company       object
dtype: object

In [17]:
duns_ENTmap = df_duns_mapped_ENT.dunsnumber.tolist()

In [18]:
df_cust= df_cust[~df_cust.dunsnumber.isin(duns_ENTmap)]

In [19]:
df_cust.shape

(2348872, 7)

In [20]:
import collections
class OrderedSet(collections.Set):
    def __init__(self, iterable=()):
        self.d = collections.OrderedDict.fromkeys(iterable)

    def __len__(self):
        return len(self.d)

    def __contains__(self, element):
        return element in self.d

    def __iter__(self):
        return iter(self.d)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [21]:
WORD = re.compile(r"\w+")

def text_to_vector(text):
    words = WORD.findall(text.upper())
    return Counter(words)

def get_cosine(vec1, vec2):
    vec1 = text_to_vector(vec1)
    vec2 = text_to_vector(vec2)
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def fuzzy(s1, s2):
    intersection = set(s1.keys()) & set(s2.keys())
    a= ' '.join(list(OrderedSet(s1.keys()) - OrderedSet(intersection)))
    b= ' '.join(list(OrderedSet(s2.keys()) - OrderedSet(intersection)))
    #print(intersection,'-' ,s1.keys(),'-', a,'-', s2.keys(),'-', b)
    if len(a)==0 or len(b)==0:
        return 100
    else:
        return fuzz.ratio(a, b)

def sim_metric(df, col1, col2):
    return SequenceMatcher(None, df[col1], df[col2]).ratio()

def minEdit(x1,x2):
    r= (fuzzy(text_to_vector(x1), text_to_vector(x2)))
    return r

1st Iteration

In [22]:
def FirstIteration(df,df_cust):
    df_first = pd.merge(df,df_cust,how='inner',
                    left_on=['company_name', 'city_name', 'state_or_province_name','iso_country_code'],
                 right_on=['company', 'city', 'state','iso_3_code'])
    df_first = df_first.drop_duplicates(subset=['gu_party_id','gu_cust_mkt_segment_cd'], keep='first')
    duns_fullymap = df_first.gu_party_id.tolist()
    df_1= df[~df.gu_party_id.isin(duns_fullymap)]
    df_first.to_csv('US_Commercial_First_Iteration_'+version+'.csv')
    return df_1,df_first

In [23]:
df_1,df_first = FirstIteration(df,df_cust)

In [24]:
df_first.shape

(33927, 18)

In [25]:
df_first.head()

,gu_party_id,gu_primary_name,duns_number,gu_cust_mkt_segment_cd,alt_dunsnumber,company_name,branch_level,iso_country_code,line_1_address,city_name,state_or_province_name,dunsnumber,country,iso_3_code,city,state,address,company
0,269682339,WARWICK PARTNERS,-999,COM,-999,WARWICK PARTNERS,BR,USA,4444 CARTER CREEK PKWY,BRYAN,TX,019224515,USA,USA,BRYAN,TX,4444 CARTER CREEK PKWY # 109,WARWICK PARTNERS
1,251365967,DATAVIDEO CORPORATION,-999,COM,-999,DATAVIDEO,HQ,USA,7048 ELMER AVE,WHITTIER,CA,830612821,USA,USA,WHITTIER,CA,7048 ELMER AVE,DATAVIDEO
2,255524598,VAN POOL TRANSPORTATION COMPANY,-999,COM,-999,VAN POOL TRANSPORTATION,BR,USA,18 COTTAGE AVE,WILBRAHAM,MA,969098318,USA,USA,WILBRAHAM,MA,70 POST OFFICE PARK # 700,VAN POOL TRANSPORTATION
3,33859033,CAPSTONE GROUP LLC,018925763,COM,-999,CAPSTONE EQUITIES,BR,USA,14 WALL ST,NEW YORK,NY,830665308,USA,USA,NEW YORK,NY,545 5TH AVE RM 1209,CAPSTONE EQUITIES
4,323188966,RECYCLE TRACK SYSTEMS INC,-999,COM,-999,RECYCLE TRACK SYSTEMS,BR,USA,435 HUDSON ST,NEW YORK,NY,080049186,USA,USA,NEW YORK,NY,435 HUDSON ST RM 404,RECYCLE TRACK SYSTEMS


In [26]:
def SecondIteration(df_1,df_cust):
    df_second = pd.merge(df_1,df_cust,how='inner',
                     left_on=['company_name','state_or_province_name','iso_country_code'],
                     right_on=['company','state','iso_3_code'])
    df_second = df_second.drop_duplicates(subset=['gu_party_id'], keep='first')   
    second_duns = df_second['gu_party_id'].unique().tolist()
    df_second.to_csv('US_Commercial_Second_Iteration_'+version+'.csv')
    df_2= df_1[~df_1.gu_party_id.isin(second_duns)]
    return df_2,df_second

In [27]:
df_2,df_second = SecondIteration(df_1,df_cust)

In [28]:
df_second.shape

(22646, 18)

In [29]:
def ThirdIteration(df_2,df_cust):
    df_third =  pd.merge(df_2,df_cust,how='inner',
                      left_on=['company_name','iso_country_code'],
                     right_on=['company','iso_3_code'])
    df_third['city_cosine'] = df_third.apply(lambda x: get_cosine(x['city_name'],x['city']), axis=1)
    df_third['city_Fuzzy'] = df_third.apply(lambda x:minEdit(x['city_name'], x['city']), axis=1)
    df_third = df_third[['gu_party_id','gu_primary_name', 'duns_number','gu_cust_mkt_segment_cd','alt_dunsnumber','branch_level',
                     'dunsnumber','iso_country_code','iso_3_code','country','company','company_name',
                     'state','state_or_province_name','address','line_1_address',
                     'city','city_name','city_cosine','city_Fuzzy']]
    df_third = df_third[(df_third['city']!='NONE')&(df_third['city_name']!='NONE')]
    df_third_40 = df_third[(df_third['city_cosine']>0.7)|
                       (df_third['city_Fuzzy']>70)|
                       ((df_third['city_cosine']>0.6)&(df_third['city_Fuzzy']>60))]
    df_third_40['city_avg']= ((df_third_40['city_cosine']*100) + df_third_40['city_Fuzzy'])/2
    df_third_40 = df_third_40[df_third_40.groupby(['gu_party_id'])['city_avg'].transform(max) == df_third_40['city_avg']]
    df_third_40.drop_duplicates(subset=['gu_party_id','city_avg'],inplace=True)
    del df_third_40['city_avg']
    df_third_40.to_csv('US_Commercial_Third_Iteration_'+version+'.csv')
    third_duns = df_third_40['gu_party_id'].unique().tolist()
    df_3= df_2[~df_2.gu_party_id.isin(third_duns)]
    return df_3,df_third_40

In [30]:
df_3,df_third = ThirdIteration(df_2,df_cust)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [31]:
df_third.shape

(1464, 20)

In [32]:
df_3.shape,df_cust.shape

((1789390, 11), (2348872, 7))

4th iteration

In [33]:
df_3.drop_duplicates(subset=['gu_party_id','company_name','gu_cust_mkt_segment_cd'],inplace=True)
del df_3['line_1_address']
del df_3['gu_cust_mkt_segment_cd']
del df_cust['address']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
df_3.shape

(1789389, 9)

In [35]:
df_3_sample= df_3[300000:400000]

In [36]:
df_fourth =  pd.merge(df_3_sample,df_cust,how='inner',
                      left_on=['state_or_province_name','city_name','iso_country_code'],
                      right_on=['state','city','iso_3_code'])

In [37]:
df_fourth.head()

,gu_party_id,gu_primary_name,duns_number,alt_dunsnumber,company_name,branch_level,iso_country_code,city_name,state_or_province_name,dunsnumber,country,iso_3_code,city,state,company
0,83318249,SPACE INTERNATIONAL LLC,789889685,789889685,SPACE INTERNATIONAL,BR,USA,MIAMI,FL,791961001,USA,USA,MIAMI,FL,CAPSTONE GROUP
1,83318249,SPACE INTERNATIONAL LLC,789889685,789889685,SPACE INTERNATIONAL,BR,USA,MIAMI,FL,831202028,USA,USA,MIAMI,FL,GLOBAL MIND USA
2,83318249,SPACE INTERNATIONAL LLC,789889685,789889685,SPACE INTERNATIONAL,BR,USA,MIAMI,FL,779953582,USA,USA,MIAMI,FL,IDEAL LIFESTYLE CONCIERGE
3,83318249,SPACE INTERNATIONAL LLC,789889685,789889685,SPACE INTERNATIONAL,BR,USA,MIAMI,FL,807827063,USA,USA,MIAMI,FL,STATETRUST LIFE AND ANNUITIES
4,83318249,SPACE INTERNATIONAL LLC,789889685,789889685,SPACE INTERNATIONAL,BR,USA,MIAMI,FL,840818350,USA,USA,MIAMI,FL,ASSOCIATION OF CERTIFIED BUSINESS FIDUCIARIES I


In [38]:
#del df_fourth['gu_cust_mkt_segment_cd']

In [39]:
df_fourth.shape

(342633457, 15)

In [40]:
total= df_fourth.shape[0]
batch = 20000000
batches = int(np.ceil(total/batch))
downrange=0
uprange = (batch*batches)+1

In [41]:
downrange,uprange,batches

(0, 360000001, 18)

In [42]:
def batch_split_df(step1,step2,batch,Dataframe):
    index_values = list(range(step1,step2,batch))
    df_list = []
    for i in range(0,len(index_values)):
        if(i==len(index_values)-1):
            break;
        else:
            print(i)
            range_val1 = index_values[i];
            range_val2 = index_values[i+1];
            start = time.process_time()
            df_new = Dataframe.iloc[range_val1:range_val2,:]
            df_new.drop_duplicates(subset=['gu_party_id','company_name','company'],inplace=True)
            df_new['cmp_cosine'] = df_new.apply(lambda x: get_cosine(x['company'],x['company_name']),axis=1)
            df_new['cmp_Fuzzy'] = df_new.apply(lambda x:minEdit(x['company'], x['company_name']), axis=1)
            df_new2 = df_new[(df_new['cmp_cosine']>0.5)|(df_new['cmp_Fuzzy']>50)]
                      # ((df_new['cmp_cosine']>0.6)&(df_new['cmp_Fuzzy']>60))]
            #df_new2 = df_new[(df_new['cmp_cosine']>0.4)&(df_new['cmp_Fuzzy']>40)]
            df_list.append(df_new2)
            print(df_new.shape)
            print(df_new2.shape)
            df_new2.to_csv('batch_data_'+str(i)+'.csv',index=False)
            #df_new.to_csv('manual_batch_data'+'clean_both'+str(i)+'.csv',index=False)
            print(time.process_time() - start)
    Final_DF = pd.concat(df_list)
    return Final_DF

In [43]:
df_fourth.shape

(342633457, 15)

In [ ]:
start = time.process_time()
df_fourth_final = batch_split_df(downrange,uprange,batch,df_fourth)
print(time.process_time() - start)

0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

(19712959, 17)
(72747, 17)
2548.6159898819997
1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

(19681118, 17)
(67485, 17)
2578.7753969789997
2


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

(19676263, 17)
(73602, 17)
2659.5721704519983
7


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

(19729043, 17)
(65319, 17)
2672.2869530109965
8


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def FourthIteration(df_fourth_final):
    df_fourth_final['space'] = (df_fourth_final.company.apply(lambda x:len(x)))  -(df_fourth_final.company.apply(lambda x:x.count(' ')))
    df_fourth_final = df_fourth_final[df_fourth_final['space']>2]
    del df_fourth_final['space']
    df_fourth_final['company_avg']= ((df_fourth_final['cmp_cosine']*100) + df_fourth_final['cmp_Fuzzy'])/2
    df_fourth_final = df_fourth_final[df_fourth_final.groupby(['gu_party_id'])['company_avg'].transform(max) == df_fourth_final['company_avg']]
    df_fourth_final.drop_duplicates(subset=['gu_party_id','company_avg'],inplace=True)
    del df_fourth_final['company_avg']
    removing_ids = df_fourth_final[(df_fourth_final['cmp_cosine']==0)&(df_fourth_final['cmp_Fuzzy']<85)]['gu_party_id'].tolist()
    df_fourth_final = df_fourth_final[~df_fourth_final.gu_party_id.isin(removing_ids)]
    df_fourth_final.to_csv('US_Commercial_Fourth_Iteration_'+version+'.csv')
    return df_fourth_final

In [ ]:
df_fourth_final_1= FourthIteration(df_fourth_final)

In [ ]:
df_fourth_final_1.shape

In [ ]:
df_fourth_final_1.to_csv('US_Commercial_Fourth_Iteration_300-400k.csv',index=False)

In [ ]:
#del df_fourth_final_1['Unnamed: 0']

In [ ]:
#df_fourth_final_1.head()

In [ ]:
df_fourth_final_1.shape

In [ ]:
#df_fourth_final_1 = df_fourth_final_1[(df_fourth_final_1['cmp_cosine']>0.7)|(df_fourth_final_1['cmp_Fuzzy']>70)|((df_fourth_final_1['cmp_cosine']>0.6)&(df_fourth_final_1['cmp_Fuzzy']>60))]

In [ ]:
#df_fourth_final_1.to_csv('US_Commercial_Fourth_Iteration_v1.csv',index=False)

Exporting OutPut Files to SnowFlake

In [ ]:
df_first.shape,df_second.shape,df_third.shape,df_fourth_final_1.shape

In [ ]:
export_table_names = ['US_Comm_1iter_match','US_Comm_2iter_match',
                      'US_Comm_3iter_match','US_Comm_4iter_match']
#

In [ ]:
#If already exist drop it
for i in export_table_names:
    print(sf_helper.execute('drop table if exists edw_dna_db.wi.'+i))

In [ ]:
df_first.to_sql(name='US_Comm_1iter_match', con=sf_engine, schema='EDW_DNA_DB.WI',index=False)
df_second.to_sql(name='US_Comm_2iter_match', con=sf_engine, schema='EDW_DNA_DB.WI',index=False)
df_third.to_sql(name='US_Comm_3iter_match', con=sf_engine, schema='EDW_DNA_DB.WI',index=False)
df_fourth_final_1.to_sql(name='US_Comm_4iter_match', con=sf_engine, schema='EDW_DNA_DB.WI',index=False)

Post Mapping

In [ ]:
# file1 = open("ent_sqlcode2.txt")
# line1 = file1.read()
# file1.close()

In [ ]:
# def PostSql(queries):
#     start = time.process_time()
#     for i in line1.strip().split(';'):
#         print(sf_helper.execute(i))
#         print(time.process_time()-start)
        
# PostSql(line1)
    